# 如何在多GPU上运行Keras模型？

我们建议使用 TensorFlow 后端来执行这项任务。有两种方法可在多个 GPU 上运行单个模型：
* 数据并行
* 设备并行

在大多数情况下，　你最需要的是数据并行

## 数据并行
数据并行包括在每个设备上复制一次目标模型，并使用每个模型副本处理不同部分的输入数据。Keras 有一个内置的实用函数 keras.utils.multi_gpu_model，它可以生成任何模型的数据并行版本，在多达 8 个 GPU 上实现准线性加速。

In [3]:
from keras.utils import multi_gpu_model

In [6]:
# 将model复制到８个GPU上
# 假定你的机器上有８个GPU可以用

parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop')

# 这个fit将调用在８个GPU上
# 由于batch_size为２５６，每个GPU将处理３２个样本
parallel_model.fit(x, y, epochs=20, batch_size=256)

## 设备并行

设备并行性包括在不同设备上运行同一模型的不同部分。对于具有并行体系结构的模型，例如有两个分支的模型，　这种方式很合适。

In [14]:
import keras
import tensorflow as tf

# 模型中共享的 LSTM 用于并行编码两个不同的序列
input_a = keras.Input(shape=(140, 256))
input_b = keras.Input(shape=(140, 256))

shared_lstm = keras.layers.LSTM(64)

# 在一个 GPU 上处理第一个序列
with tf.device('/gpu:0'):
    encoded_a = shared_lstm(tweet_a)
# 在另一个 GPU上 处理下一个序列
with tf.device('/gpu:1'):
    encoded_b = shared_lstm(tweet_b)

# 在 CPU 上连接结果
with tf.device_scope('/cpu:0'):
    merged_vector = keras.layers.concatenate([encoded_a, encoded_b],
                                             axis=-1)